In [ ]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
from opencage.geocoder import OpenCageGeocode

In [ ]:
opencagekey = open('key.secret', 'r')
key = opencagekey.read().replace('\n', '')
opencagekey.close()

In [ ]:
dataset = pd.read_csv('caso_full.csv')

In [ ]:
print(dataset[0:15])

In [ ]:
dataset = pd.read_csv('caso_full.csv', encoding = 'utf-8', usecols=[\
    'city',
    'city_ibge_code',
    'date',
    'epidemiological_week',
    'estimated_population',
    'last_available_confirmed',
    'last_available_confirmed_per_100k_inhabitants',
    'last_available_deaths',
    'place_type',
    'state',
    'new_confirmed',
    'new_deaths'])

In [ ]:
print(dataset[0:15])


In [ ]:
dataset = dataset.rename(columns = {\
    'city':                     'muni',
    'city_ibge_code':           'codIbge',
    'date':                     'data',
    'epidemiological_week':     'semEpid',
    'estimated_population':     'popEstim',
    'last_available_confirmed': 'confAcc',
    'last_available_confirmed_per_100k_inhabitants': 'confAcc100k',
    'last_available_deaths':    'obitoAcc',
    'place_type':               'tipoLocal',
    'state':                    'uf',
    'new_confirmed':            'confDia',
    'new_deaths':               'obitoDia'})

In [ ]:
print(dataset.head(15))

In [ ]:
dataset.dropna(subset = ['muni', 'codIbge'], inplace = True)

In [ ]:
print(dataset.head(15))


In [ ]:
aggregation = {
    'muni':        'first', # Primeiro valor encontrado
    'codIbge':     'first', # Primeiro valor encontrado
    'data':        'last',  # Ultimo valor encontrado
    'semEpid':     'first', # Primeiro valor encontrado
    'popEstim':    'max',   # Maximo valor do conjunto
    'confAcc':     'max',   # Maximo valor do conjunto
    'confAcc100k': 'max',   # Maximo valor do conjunto
    'obitoAcc':    'max',   # Maximo valor do conjunto
    'tipoLocal':   'first', # Primeiro valor encontrado
    'uf':          'first', # Primeiro valor encontrado
    'confDia':     'sum',   # Soma dos valores do conjunto
    'obitoDia':    'sum'}   # Soma dos valores do conjunto
# Agrega resultados da semana epidemiológica por município
dataset = dataset.groupby(['semEpid', 'codIbge']).aggregate(aggregation)\
                 .reindex(columns = dataset.columns)

In [ ]:
print(dataset.head(15))

In [ ]:
dataset.reset_index(drop = True, inplace = True)

In [ ]:
print(dataset.head(15))


In [ ]:
print(dataset.shape[0])

In [ ]:
if not os.path.exists('rocketseat.csv'):
    colNames = ['muni', 'codIbge', 'uf', 'lat', 'lon']
    geoloc = pd.DataFrame(columns = colNames)
    geoloc.to_csv('rocketseat.csv', encoding = 'utf-8')

In [ ]:
geoloc = pd.read_csv('teste1.csv', encoding = 'utf-8', usecols=['muni', 'codIbge', 'uf', 'lat', 'lon'])

In [ ]:
print(geoloc.head(15))

In [ ]:
joinDataset = pd.merge(dataset, geoloc, on = ['muni', 'codIbge', 'uf'], how = 'left', indicator = True)

In [ ]:
print(joinDataset.head(15))

In [ ]:
print(joinDataset.loc[joinDataset['_merge'] == 'left_only'])

In [ ]:
print(joinDataset[875:885])

In [ ]:
getgeo = []
for index, rows in joinDataset.iterrows():
    if (rows._merge == 'left_only') and ([rows.muni, rows.codIbge, rows.uf] not in getgeo):
        getgeo.append([rows.muni, rows.codIbge, rows.uf])
        query = (str(rows.muni.lower()) + ', ' + str(rows.uf.lower()) + ', ' + 'brasil')
        geocoder = OpenCageGeocode(key)
        coord = geocoder.geocode(query)
        geoloc = geoloc.append(pd.Series([
            rows.muni,
            rows.codIbge,
            rows.uf,
            coord[0]['geometry']['lat'],
            coord[0]['geometry']['lng']], index = geoloc.columns), ignore_index = True)
        print('Municipio adicionado: ', rows.muni, rows.codIbge, rows.uf, coord[0]['geometry']['lat'], coord[0]['geometry']['lng'])
        joinDataset['lat'] = coord[0]['geometry']['lat']
        joinDataset['lon'] = coord[0]['geometry']['lng']


In [ ]:
print(len(getgeo))

In [ ]:
print(joinDataset.head(15))


In [ ]:
print(joinDataset[875:885])

In [ ]:
if len(getgeo) > 0:
    geoloc.to_csv('muniset.csv', encoding = 'utf-8')

In [ ]:
print(getgeo)

In [ ]:
joinDataset.to_csv('covid_dataset.csv', encoding = 'utf-8')